# SPT analysis - Interactive report
###### AB @ BCB ITQB 


### Instructions:
1. To run a cell select it and press **SHIFT+ENTER**;
2. All cells require the **previous** one to be run first;
3. If you change the loaded file through the dropdown menu, **RUN** the interactive graph cell (the next one) again to update the data 
4. If you make a mistake in the order or just want to restart the report **restart the kernel** and start from the beginning
---

### Import required functions

In [ ]:
%matplotlib widget
from ReportJupyter import *

### Choose folder that contains all npy's

In [ ]:
# Choose folder
root = Tk()
rootdir = filedialog.askdirectory()
root.destroy()

load_widget = interactive(FileLoader, 
                          folder=Dropdown(options=os.listdir(rootdir), description='Experimental conditions:', disabled=False), 
                          root = fixed(rootdir))
display(load_widget)

### Interactive filtering
- Filter the tracks by assigning a maximum angle;
- This angle refers to the deviation of the Z-ring ellipse to a perfect circle
- Higher angles correspond to slanted rings and higher errors

In [ ]:
TrackObjects = np.array(load_widget.result)
angle = np.rad2deg(np.arccos([i.ellipse['minor']/i.ellipse['major'] for i in TrackObjects]))

angle_wid = interactive(Update_Graphs, 
            angle_threshold= IntSlider(min=1, max=91, step=2, value=20), 
            all_tracks=fixed(TrackObjects), 
            angles=fixed(angle))

accordion = Accordion(children=[angle_wid], titles=('Filter by angle'))
accordion
